In [8]:
import os
import torch
from torch import nn
import torchtext
from torchtext.data import get_tokenizer
import numpy as np
import pandas as pd
import json
from torch.utils.data import Dataset , DataLoader

In [9]:
tokenizer = get_tokenizer('spacy')
text = "This is a pytorch U.S.A"
tokens = tokenizer(text)
print(tokens)

['This', 'is', 'a', 'pytorch', 'U.S.A']


In [11]:
tag_list=["ineffective","unnecessary","pharma","rushed","side-effect","mandatory","country","ingredients","political","none","conspiracy","religious"]
tag_to_idx={}
for i in range(len(tag_list)):
    tag_to_idx[tag_list[i]]=i

ineffective 0
unnecessary 1
pharma 2
rushed 3
side-effect 4
mandatory 5
country 6
ingredients 7
political 8
none 9
conspiracy 10
religious 11


In [ ]:
def load_data(path)
    with open(path,'r',encoding='utf-8') as file:
        data = json.load(file)
    # print(len(train_data))
    return data

In [10]:


class CustomDataset(Dataset):
    def __init__(self,data, label):
        self.data = data 
        self.label = label

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index) :
        return self.data[index] , self.label[index]
 


In [12]:
embed_len = 50 
batch_size = 32
num_label=len(tag_list)

def vectorize(label)

6956


In [ ]:
tmp =load_data("train.json")
print(len(tmp))
print(tmp[0])